## How to generate exact quasiprobability distributions from Sampler

This how-to guide is intended to show users how they can generate statevector-based quasiprobability distributions through the `qiskit.primitives.BaseSampler` interface.

In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import PauliList

from circuit_knitting.cutting import (
    partition_problem,
    execute_experiments,
)

Prepare inputs to `execute_experiments`

In [2]:
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
observables = PauliList(["ZZ"])
subcircuits, _, subobservables = partition_problem(
    circuit=circuit, partition_labels="AB", observables=observables
)

In order to calculate exact quasiprobability distributions for circuits with mid-circuit measurements, users will need to use the `ExactSampler` class from `circuit_knitting.utils.simulation`. The Qiskit Samplers do not support mid-circuit measurements in statevector mode.

In [3]:
from circuit_knitting.utils.simulation import ExactSampler

exact_sampler = ExactSampler()

If `ExactSampler` is used, the quasiprobability distributions returned from `execute_experiments` will be exact and generated from the statevectors of the subexperiments.

In [4]:
quasi_dists, coefficients = execute_experiments(
    circuits=subcircuits,
    subobservables=subobservables,
    num_samples=1000,
    samplers=exact_sampler,
)